In [4]:
import os
import pandas as pd
from data_preprocessing import preprocess_data
from chunking import chunk_data
from storing_retrieval import index_chunks, retrieve_similar_chunks
from question_generation import generate_sub_questions
from response_generation import generate_final_response
from evaluation import evaluate_response_with_ragas

In [5]:
# Define test parameters
company_name = "AXP"  # You can loop through different companies if needed
document_type = "10K"
chunking_methods = ["Fixed-size Chunking", "Sentence-based Chunking"]
queries = {
    "Broad Financial Question": [
        "What is the growth prospects of the company?",
        "Given the current circumstances, will the company be able to outperform its competitors in the next 12 months?"
    ],
    "Financial Foundation": [
        "What are some key financial ratios of the company?",
        "How stable is the profit margin outlook?"
    ]
}

In [6]:
# File path setup
data_dir = "data"
folder = "10Ks" if document_type == "10K" else "reports"
file_suffix = "-2024-10k.pdf" if document_type == "10K" else "-report.pdf"
file_path = os.path.join(data_dir, folder, f"{company_name.lower()}{file_suffix}")

In [13]:
def evaluate_rag(chunking_method, query_type, user_query, use_sub_questions=False):
    try:
        print(f"\nTesting for Query Type: {query_type} | Chunking Method: {chunking_method} | Sub-questions: {use_sub_questions}")
        
        # Step 1: Preprocess the document
        with open(file_path, "rb") as file:
            raw_text = preprocess_data(file)

        # Step 2: Chunk the document
        print("Chunking the document...")
        chunks = chunk_data(raw_text, chunking_method)
        print(f"Number of chunks created: {len(chunks)}")

        # Step 3: Store chunks in ChromaDB
        collection_name = f"{company_name}_{document_type}_{chunking_method}".lower().replace(" ", "_")
        print("Indexing chunks into ChromaDB...")
        index_chunks(chunks, collection_name)

        # Step 4: Process query
        if use_sub_questions:
            print("Generating sub-questions...")
            sub_questions = generate_sub_questions(user_query)
            print(f"Sub-Questions Generated: {sub_questions}")
        else:
            sub_questions = [user_query]

        # Step 5: Retrieve relevant chunks
        print("Retrieving relevant chunks...")
        aggregated_chunks = []
        for question in sub_questions:
            retrieved_chunks, retrieved_indices, distances = retrieve_similar_chunks(question, collection_name, top_k=5)
            aggregated_chunks.extend(retrieved_chunks)
        
        print(f"Total Retrieved Chunks: {len(aggregated_chunks)}")
        avg_cosine_similarity = sum(distances) / len(distances) if distances else 0

        # Step 6: Generate final response
        print("Generating final response...")
        final_response = generate_final_response(aggregated_chunks, user_query)
        print(f"Final Response: {final_response}")

        # Step 7: Evaluate the response using RAGAs
        print("Evaluating response with RAGAs...")
        evaluation_results_df = evaluate_response_with_ragas(
            questions=[user_query],
            answers=[final_response],
            contexts=[aggregated_chunks],
        )

        # Return evaluation results
        return {
            "Total Chunks": len(chunks),
            "Retrieved Chunks": len(aggregated_chunks),
            "Average Cosine Similarity": avg_cosine_similarity,
            "Evaluation Results": evaluation_results_df,
            "Sub-questions Used": use_sub_questions
        }

    except Exception as e:
        print(f"Error during pipeline execution: {e}")
        return None


In [14]:
# Main testing loop
results = []
for chunking_method in chunking_methods:
    for query_type, query_list in queries.items():
        for query in query_list:
            # Simple RAG (no sub-questions)
            result_simple = evaluate_rag(chunking_method, query_type, query, use_sub_questions=False)
            if result_simple:
                results.append({
                    "Approach": "Simple RAG",
                    "Query Type": query_type,
                    "Chunking": chunking_method,
                    **result_simple
                })

            # Multi-Layer RAG (with sub-questions)
            result_multi_layer = evaluate_rag(chunking_method, query_type, query, use_sub_questions=True)
            if result_multi_layer:
                results.append({
                    "Approach": "Multi-Layer RAG",
                    "Query Type": query_type,
                    "Chunking": chunking_method,
                    **result_multi_layer
                })

# Save results to a CSV for the table
results_df = pd.DataFrame(results)
results_csv_path = "evaluation_results_rag.csv"
results_df.to_csv(results_csv_path, index=False)

# Print results
print("\nFinal Evaluation Results:")
print(results_df)
print(f"Results saved to {results_csv_path}")


Testing for Query Type: Broad Financial Question | Chunking Method: Fixed-size Chunking | Sub-questions: False
Chunking the document...
Number of chunks created: 165
Indexing chunks into ChromaDB...


Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Insert of existing embedding ID: 9
Insert of existing embedding ID: 10
Insert of existing embedding ID: 11
Insert of existing embedding ID: 12
Insert of existing embedding ID: 13
Insert of existing embedding ID: 14
Insert of existing embedding ID: 15
Insert of existing embedding ID: 16
Insert of existing embedding ID: 17
Insert of existing embedding ID: 18
Insert of existing embedding ID: 19
Insert of existing embedding ID: 20
Insert of existing embedding ID: 21
Insert of existing embedding ID: 22
Insert of existing embedding ID: 23
Insert of existing embedding ID: 24
Insert of existing embedding ID: 25
Insert of existing embedding ID: 26
Insert of existing embedding ID: 27
In

Retrieving relevant chunks...
Total Retrieved Chunks: 5
Generating final response...
Final Response: The growth prospects of American Express Company appear robust, supported by several key factors outlined in their latest SEC 10-K report. 

1. **Billed Business Growth**: The company experienced a **9% year-over-year increase** in billed business, which is identified as the most significant driver of their financial results. Notably, the first quarter showed particularly strong growth, benefiting from the prior year's challenges posed by the Omicron variant. Although there was a softer spending environment towards the end of the year, the growth in key customer segments, particularly among Millennial and Gen-Z Card Members, is promising for future growth.

2. **Revenue Growth**: Total revenues net of interest expense increased **14% year-over-year**, with all revenue lines contributing positively. The growth in billed business led to a **9% rise in Discount revenue**, the largest reven

Evaluating: 100%|██████████| 3/3 [00:24<00:00,  8.15s/it]



Testing for Query Type: Broad Financial Question | Chunking Method: Fixed-size Chunking | Sub-questions: True
Chunking the document...
Number of chunks created: 165
Indexing chunks into ChromaDB...


...


Testing for Query Type: Financial Foundation | Chunking Method: Sentence-based Chunking | Sub-questions: True
Chunking the document...
Number of chunks created: 170
Indexing chunks into ChromaDB...


Insert of existing embedding ID: 0
Insert of existing embedding ID: 1
Insert of existing embedding ID: 2
Insert of existing embedding ID: 3
Insert of existing embedding ID: 4
Insert of existing embedding ID: 5
Insert of existing embedding ID: 6
Insert of existing embedding ID: 7
Insert of existing embedding ID: 8
Insert of existing embedding ID: 9
Insert of existing embedding ID: 10
Insert of existing embedding ID: 11
Insert of existing embedding ID: 12
Insert of existing embedding ID: 13
Insert of existing embedding ID: 14
Insert of existing embedding ID: 15
Insert of existing embedding ID: 16
Insert of existing embedding ID: 17
Insert of existing embedding ID: 18
Insert of existing embedding ID: 19
Insert of existing embedding ID: 20
Insert of existing embedding ID: 21
Insert of existing embedding ID: 22
Insert of existing embedding ID: 23
Insert of existing embedding ID: 24
Insert of existing embedding ID: 25
Insert of existing embedding ID: 26
Insert of existing embedding ID: 27
In

Generating sub-questions...
Sub-Questions Generated: ["What factors have influenced the company's profit margins over the past few years as detailed in the 10-K document?", "How do the company's projected profit margins compare to industry averages and competitors, as mentioned in the SEC filings?", "What specific risks or challenges outlined in the 10-K could potentially impact the stability of the company's profit margins?", "How has the company's cost structure changed, and what implications does this have for future profit margins according to the 10-K?", 'What trends in revenue growth or decline are noted in the 10-K, and how might these affect profit margin stability?', 'Are there any management strategies or initiatives discussed in the 10-K that aim to enhance or stabilize profit margins in the future?']
Retrieving relevant chunks...
Total Retrieved Chunks: 30
Generating final response...
Final Response: The profit margin outlook for American Express appears moderately stable, 

Evaluating: 100%|██████████| 3/3 [00:35<00:00, 11.67s/it]



Final Evaluation Results:
           Approach                Query Type                 Chunking  \
0        Simple RAG  Broad Financial Question      Fixed-size Chunking   
1   Multi-Layer RAG  Broad Financial Question      Fixed-size Chunking   
2        Simple RAG  Broad Financial Question      Fixed-size Chunking   
3   Multi-Layer RAG  Broad Financial Question      Fixed-size Chunking   
4        Simple RAG      Financial Foundation      Fixed-size Chunking   
5   Multi-Layer RAG      Financial Foundation      Fixed-size Chunking   
6        Simple RAG      Financial Foundation      Fixed-size Chunking   
7   Multi-Layer RAG      Financial Foundation      Fixed-size Chunking   
8        Simple RAG  Broad Financial Question  Sentence-based Chunking   
9   Multi-Layer RAG  Broad Financial Question  Sentence-based Chunking   
10       Simple RAG  Broad Financial Question  Sentence-based Chunking   
11  Multi-Layer RAG  Broad Financial Question  Sentence-based Chunking   
12       Si